This notebook takes a list of question ids from a Stack Exchange site like Stack Overflow and returns all the answers to those questions. It requires a Stack Exchange API key. One can be acquired at https://api.stackexchange.com

In [ ]:
import requests
import json
import csv

# Input API key and question ids
#api_key = ‘REPLACE WITH STACK EXCHANGE API KEY’
#stack_exchange_community = ‘REPLACE WITH STACK EXCHANGE COMMUNITY e.g. stackoverflow’
#question_ids = ‘REPLACE WITH LIST OF COMMA-SEPARATED QUESTION IDS’


def get_answers(question_ids, stack_exchange_community, api_key):
    answers = {}
    for question_id in question_ids:
        url = f"https://api.stackexchange.com/2.2/questions/{question_id}/answers?order=desc&sort=activity&site={stack_exchange_community}&filter=withbody&key={api_key}"
        response = requests.get(url)
        data = json.loads(response.text)
        if 'items' in data and len(data['items']) > 0:
            answers[question_id] = []
            for item in data['items']:
                if 'body' in item:
                    user = item['owner']['display_name'] if 'owner' in item else 'Unknown'
                    score = item['score'] if 'score' in item else 0
                    accepted = item['is_accepted'] if 'is_accepted' in item else False
                    answer_text = item['body']
                    answer = f"User: {user}\nScore: {score}\nAccepted: {accepted}\n\n\n{answer_text}"
                    answers[question_id].append(answer)
        elif 'error_message' in data:
            print(f"Error: {data['error_message']}")
        else:
            answers[question_id] = "No answers found"
        print(response.text)
    return answers

answers = get_answers(question_ids, stack_exchange_community, api_key)

if answers:
    # create a csv file
    with open('answers.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Question ID', 'Answer'])
        for question_id in answers:
            for answer in answers[question_id]:
                writer.writerow([question_id, answer])
    print("Answers saved to answers.csv")
else:
    print("No answers found")